# Analysing the piecewise voltage simulations (concatenated elsewhere)

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
import math
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c
import flopter.spice.inputparser as inp
import flopter.spice.normalise as nrm
import flopter.spice.utils as spu

# Preamble of necessary stuff

This bit creates all teh necessary variables for plotting to commence, including loading the dataset, creating a denormaliser, extracting relevant probe dimensions from an input file and defining a few functions for ease of manipulation. 

In [2]:
spice_dir = pth.Path('/home/jleland/data/external_big/spice/')
os.chdir(spice_dir)

In [3]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [4]:
cb_palette = ['#377eb8', '#ff7f00', '#4daf4a', 
              '#f781bf', '#a65628', '#984ea3', 
              '#999999', '#e41a1c', '#dede00',
              '#377eb8', '#ff7f00', '#4daf4a', 
              '#f781bf', '#a65628', '#984ea3', 
              '#999999', '#e41a1c', '#dede00']

In [5]:
skippable_scans = set([
    'marconi/spice2/piecewise/flat_flush_piecewise',
    'marconi/spice2/piecewise/flat_recessed_10',
    'marconi/spice2/piecewise/flat_recessed_10_1_',
    'marconi/spice2/piecewise/flat_recessed_8-12',
]) 
single_sims = set()

## Load datasets!

In [6]:
piecewise_ds = xr.load_dataset('piecewise_sheath_exp_ivs.nc')
piecewise_ds

<xarray.Dataset>
Dimensions:         (probe: 1, theta: 7, voltage: 14, y: 557, z: 233)
Coordinates:
  * probe           (probe) object 'flat_flush'
  * z               (z) int64 0 1 2 3 4 5 6 7 ... 226 227 228 229 230 231 232
  * voltage         (voltage) float64 -25.0 -18.0 -12.0 -10.0 ... -2.0 -1.0 0.0
  * y               (y) int64 0 1 2 3 4 5 6 7 ... 550 551 552 553 554 555 556
  * theta           (theta) float64 3.0 5.0 15.0 20.0 12.0 10.0 8.0
Data variables:
    current         (probe, theta, voltage) float64 10.47 10.23 ... -184.3
    d_current       (probe, theta, voltage) float64 3.618 3.592 ... 13.93 16.11
    derr_current    (probe, theta, voltage) float64 0.007561 ... 0.03366
    current_i       (probe, theta, voltage) float64 10.47 10.23 ... 14.75 10.22
    d_current_i     (probe, theta, voltage) float64 3.618 3.592 ... 3.924 3.235
    derr_current_i  (probe, theta, voltage) float64 0.007561 ... 0.006761
    current_e       (probe, theta, voltage) float64 -3.1e-06 ... -194.6
    d_current_e     (probe, theta, voltage) float64 0.001761 0.003295 ... 15.97
    derr_current_e  (probe, theta, voltage) float64 3.68e-06 ... 0.03337
    rho             (probe, theta, voltage, z, y) float64 0.0 0.0 ... 111.5
    pot             (probe, theta, voltage, z, y) float64 0.0 0.0 ... 0.0 0.0
    delta           (probe, theta, voltage) int64 32 26 19 16 12 ... 5 2 3 1 0
    delta_w         (probe, theta, voltage) int64 32 26 19 16 12 ... 5 2 3 1 0
    theta_rads      (probe, theta) float64 -0.05236 -0.08727 ... -0.1745 -0.1396
    theta_p_rads    (probe) object 'flat_flush'
    recession       (probe) object 'flat_flush'

In [7]:
old_swept_ds = xr.load_dataset('sheath_exp_fwp_ivs.nc')
swept_ds = xr.load_dataset('sheath_exp_hg_ivs.nc')
swept_ds

<xarray.Dataset>
Dimensions:               (theta: 15, voltage: 1201)
Coordinates:
  * voltage               (voltage) float64 -29.98 -29.95 ... 1.665e-15 0.025
  * theta                 (theta) float64 2.0 3.0 4.0 5.0 ... 15.0 20.0 30.0
    theta_p               float64 0.0
    theta_p_rads          float64 0.0
    theta_rads            (theta) float64 0.03491 0.05236 ... 0.3491 0.5236
Data variables:
    str_iv_chi2           (theta) float64 723.8 802.2 ... 1.07e+03 1.067e+03
    str_iv_reduced_chi2   (theta) float64 0.6514 0.7195 0.738 ... 0.9598 0.9566
    str_iv_I_sat          (theta) float64 -14.44 -21.82 -29.39 ... -108.4 -150.9
    str_iv_d_I_sat        (theta) float64 0.01673 0.02617 ... 0.1702 0.2316
    str_iv_a              (theta) float64 0.005932 0.005801 ... 0.01262 0.008279
    str_iv_d_a            (theta) float64 0.0001485 0.0001528 ... 0.0001997
    str_iv_T_e            (theta) float64 1.113 1.099 1.089 ... 1.082 1.076
    str_iv_d_T_e          (theta) float64 0.005743 0.005632 ... 0.005819 0.00541
    str_iv_V_f            (theta) float64 -2.368 -2.338 -2.332 ... -2.666 -2.736
    str_iv_d_V_f          (theta) float64 0.003084 0.002892 ... 0.002625
    str_iv_run_dir        (theta) object 'marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-2.0' ... 'marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-30.0'
    str_iv_voltage_min    (theta) float64 -29.98 -29.98 -29.98 ... -29.98 -29.98
    str_iv_voltage_max    (theta) float64 -2.125 -2.025 -2.025 ... -2.025 -2.025
    norm_iv_chi2          (theta) float64 1.106e+03 1.085e+03 ... 1.176e+03
    norm_iv_reduced_chi2  (theta) float64 1.003 0.9827 0.9814 ... 1.051 1.081
    norm_iv_I_sat         (theta) float64 -14.2 -21.51 -29.0 ... -107.3 -149.3
    norm_iv_d_I_sat       (theta) float64 0.01626 0.02425 ... 0.1448 0.2018
    norm_iv_a             (theta) float64 0.007396 0.007055 ... 0.01315 0.009172
    norm_iv_d_a           (theta) float64 0.0001542 0.0001512 ... 0.0001874
    norm_iv_voltage_min   (theta) float64 -27.61 -27.64 -27.65 ... -27.29 -27.25
    norm_iv_voltage_max   (theta) float64 -0.01163 -0.0166 ... -0.01911 -0.0208
    ion_chi2              (theta) float64 528.0 593.0 568.0 ... 820.2 776.8
    ion_reduced_chi2      (theta) float64 0.6118 0.6864 0.6566 ... 0.9649 0.916
    ion_I_sat             (theta) float64 14.46 21.91 29.43 ... 108.5 150.8
    ion_d_I_sat           (theta) float64 0.01913 0.03042 ... 0.2098 0.2797
    ion_a                 (theta) float64 0.005252 0.004938 ... 0.01178 0.007918
    ion_d_a               (theta) float64 0.0001623 0.0001699 ... 0.0002357
    ion_voltage_min       (theta) float64 3.839 3.842 3.834 ... 3.843 3.844
    ion_voltage_max       (theta) float64 12.05 12.06 12.06 ... 11.94 11.93
    current               (theta, voltage) float64 -15.88 -15.98 ... -0.0 nan
    current_e             (theta, voltage) float64 0.0 0.0 0.0 ... 0.0 nan
    current_i             (theta, voltage) float64 15.88 15.98 16.02 ... 0.0 nan
    d_current             (theta, voltage) float64 0.1944 0.2147 ... 0.0 nan
    d_current_e           (theta, voltage) float64 0.0 0.0 0.0 ... 7.594 0.0 nan
    d_current_i           (theta, voltage) float64 0.1944 0.2147 ... 0.0 nan
    time                  (theta, voltage) float64 0.0 0.06436 ... 5.294 nan

## Unleash the plotting

In [8]:
piecewise_ds.sel(probe='flat_flush').delta.plot.line(x='voltage', hue='theta')

In [9]:
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True, figsize=[10,6])

for i, theta in enumerate([8.0, 10.0, 12.0]):
    ax = axes[i]
    fixed_plot_ds = piecewise_ds.sel(voltage=slice(-25.0,0), theta=theta)
    osweep_plot_ds = old_swept_ds.sel(probe='flat_flush', theta=theta, voltage=slice(-10,0))

    (-fixed_plot_ds['current']).plot.line(x='voltage', ax=ax, label='piecewise', linestyle='none', marker='x')
    osweep_plot_ds['current'].plot.line(x='voltage', ax=ax, label=r'swept - $V_{wall}$ floating')

    ax.axhline(y=0, **c.AX_LINE_DEFAULTS)
    ax.set_title(r'$\theta = {}$'.format(theta))
    ax.legend()

fig.tight_layout()

In [10]:
print(old_swept_ds.theta.values)
print(piecewise_ds.sortby('theta').theta.values)
ovl_thetas = set(piecewise_ds.theta.values).intersection(set(old_swept_ds.theta.values))

[ 3.  4.  5.  6.  8. 10. 12. 20. 30.]
[ 3.  5.  8. 10. 12. 15. 20.]


In [11]:
fig, axes = plt.subplots(2, 3, sharex=True, sharey=True, figsize=[10,8])

for i, theta in enumerate(ovl_thetas):
    ax = axes[i//3][i%3]
    fixed_plot_ds = piecewise_ds.sel(voltage=slice(-25.0,-4.0), theta=theta)

    if theta in swept_ds.theta:
        sweep_plot_ds = old_swept_ds.sel(probe='flat_flush', theta=theta, voltage=slice(-25.0,-4.0))
        sweep_plot_ds['current'].plot.line(x='voltage', ax=ax, label=r'swept - $V_{wall}$ floating')


    (-fixed_plot_ds['current']).plot.line(x='voltage', ax=ax, label='piecewise', linestyle='none', marker='x')
    ax.axhline(y=0, **c.AX_LINE_DEFAULTS)
    ax.set_title(r'$\theta = {}$'.format(theta))
#     ax.set_ylim(None, 0)
    ax.legend()

fig.tight_layout()

In [12]:
## Make floating potential, corrected voltage and child-Langmuir voltage part of the piecewise ds

floating_pots = []
v_corrs = []
v_cls = []
# piecewise_ds.sel(probe='flat_flush', voltage=slice(-30,-4))['current_i'].plot.line(x='voltage', hue='theta', ax=ax)

for theta in piecewise_ds.theta.values:
    pw_plot_ds = piecewise_ds.sel(probe='flat_flush', theta=theta).mean(['y', 'z'])
    
    V_f = pw_plot_ds.swap_dims({'voltage':'current'}).interp(current=0)['voltage']
    floating_pots.append(V_f)    
    
    voltage_corr = pw_plot_ds['voltage'].values - V_f.values
    voltage_cl = np.float_power(np.abs(voltage_corr), 0.75)
    
    v_corrs.append(voltage_corr)
    v_cls.append(voltage_cl)
    
piecewise_ds = piecewise_ds.assign({
    'voltage_corr': (['theta', 'voltage'], v_corrs), 
    'voltage_cl': (['theta', 'voltage'], v_cls), 
    'V_f': (['theta'], floating_pots)
})

In [13]:
fig, ax = plt.subplots(1, 2, sharey=True, figsize=[8,6])

se_fitter = fts.IonCurrentSEFitter()
thetas = []
seps = []
d_seps = []
for theta in piecewise_ds.theta.values:
    
#     fixed_plot_ds = piecewise_ds.sel(theta=theta, probe='flat_flush')
    pw_plot_ds = piecewise_ds.sel(probe='flat_flush', voltage=slice(-30,-7), theta=theta)
    
    ax[0].errorbar(pw_plot_ds.voltage_corr, pw_plot_ds.current_i, yerr=pw_plot_ds.d_current_i, 
                   label=r'$\theta=${}'.format(theta))
    ax[0].set_ylabel(r'$I_i$')
    ax[0].set_xlabel(r'$V_p$')
    ax[0].legend()
    
    ax[1].plot(pw_plot_ds.voltage_cl, pw_plot_ds.current_i)
    fit_data = se_fitter.fit(pw_plot_ds.voltage_cl, pw_plot_ds.current_i, sigma=pw_plot_ds.d_current_i)
    ax[1].plot(*fit_data.get_fit_plottables(), label='a={:.3g}'.format(fit_data.get_sheath_exp()))
    ax[1].legend()
    ax[1].set_xlabel(r'$|V_p - V_f|^{\frac{3}{4}}$')
    
    thetas.append(theta)
    seps.append(fit_data.get_sheath_exp())
    d_seps.append(fit_data.get_sheath_exp_err())
    
fig.tight_layout()

In [14]:
piecewise_ds['ion_a'] = (piecewise_ds['theta'] * 0) + np.array(seps)
piecewise_ds['ion_d_a'] = (piecewise_ds['theta'] * 0) + np.array(d_seps)

In [15]:
fig, ax = plt.subplots()
piecewise_ds.sortby('theta')['ion_a'].plot(x='theta', marker='x', ls='none')
old_swept_ds.sel(probe='flat_flush')['ion_a'].plot(x='theta', marker='x', ls='none')

## Check the sheath size as a function of angle and voltage

Recreating Bergmann's figures from 1994 paper

In [16]:
piecewise_ds = piecewise_ds.sortby('theta')
piecewise_ds

<xarray.Dataset>
Dimensions:         (probe: 1, theta: 7, voltage: 14, y: 557, z: 233)
Coordinates:
  * probe           (probe) object 'flat_flush'
  * z               (z) int64 0 1 2 3 4 5 6 7 ... 226 227 228 229 230 231 232
  * voltage         (voltage) float64 -25.0 -18.0 -12.0 -10.0 ... -2.0 -1.0 0.0
  * y               (y) int64 0 1 2 3 4 5 6 7 ... 550 551 552 553 554 555 556
  * theta           (theta) float64 3.0 5.0 8.0 10.0 12.0 15.0 20.0
Data variables:
    current         (probe, theta, voltage) float64 10.47 10.23 ... -467.2
    d_current       (probe, theta, voltage) float64 3.618 3.592 ... 30.92 42.67
    derr_current    (probe, theta, voltage) float64 0.007561 ... 0.08917
    current_i       (probe, theta, voltage) float64 10.47 10.23 ... 33.17 25.92
    d_current_i     (probe, theta, voltage) float64 3.618 3.592 ... 6.269 5.519
    derr_current_i  (probe, theta, voltage) float64 0.007561 ... 0.01153
    current_e       (probe, theta, voltage) float64 -3.1e-06 ... -493.1
    d_current_e     (probe, theta, voltage) float64 0.001761 0.003295 ... 44.07
    derr_current_e  (probe, theta, voltage) float64 3.68e-06 ... 0.09209
    rho             (probe, theta, voltage, z, y) float64 0.0 0.0 ... 115.8
    pot             (probe, theta, voltage, z, y) float64 0.0 0.0 ... 0.0 0.0
    delta           (probe, theta, voltage) int64 32 26 19 16 12 ... 9 9 3 4 1
    delta_w         (probe, theta, voltage) int64 32 26 19 16 12 ... 9 9 3 4 1
    theta_rads      (probe, theta) float64 -0.05236 -0.08727 ... -0.2618 -0.3491
    theta_p_rads    (probe) object 'flat_flush'
    recession       (probe) object 'flat_flush'
    voltage_corr    (theta, voltage) float64 -20.47 -13.47 ... 2.407 3.407
    voltage_cl      (theta, voltage) float64 9.625 7.033 4.52 ... 1.932 2.508
    V_f             (theta) float64 -4.526 -4.538 -4.224 ... -3.826 -3.407
    ion_a           (theta) float64 0.01209 0.01124 0.01525 ... 0.01998 0.01519
    ion_d_a         (theta) float64 0.0007111 0.0009088 ... 0.0003731 0.0009753

In [38]:
# Figure 9A


sl_fitter = fts.StraightLineFitter()

fig, ax = plt.subplots()

delta_0s = []
d_delta_0s = []

for theta in piecewise_ds.theta.values:
    pw_plot_ds = piecewise_ds.sel(theta=theta, probe='flat_flush').swap_dims({'voltage':'voltage_corr'}).sel(voltage_corr=slice(-25, 0))
    
#     ax.plot(pw_plot_ds.voltage_cl, pw_plot_ds.delta.values, 'x', label=r'$\theta = {}$'.format(theta))
    
    delta_fit_data = sl_fitter.fit(pw_plot_ds.voltage_cl.values, pw_plot_ds.delta.values)
    ax.plot(*delta_fit_data.get_fit_plottables(), 
            label=r'$\Delta_0 = {:.3g}, \Delta_1 = {:.3g}$'.format(delta_fit_data.get_param('y_0'), 
                                                                   delta_fit_data.get_param('m')))
    ax.legend()
    
    delta_0s.append(delta_fit_data.get_param('y_0'))
    d_delta_0s.append(delta_fit_data.get_param_err('y_0'))

ax.set_ylabel(r'$\Delta$')
ax.set_xlabel(r'$|V|^{3/4}$')

delta_0s = np.array(delta_0s)
d_delta_0s = np.array(d_delta_0s)

In [29]:
# Figure 9B

fig, ax = plt.subplots(figsize=[6,6])

delta_0s = []

for i, theta in enumerate(piecewise_ds.theta.values):
    pw_plot_ds = piecewise_ds.sel(theta=theta, probe='flat_flush').swap_dims({'voltage':'voltage_corr'}).sel(voltage_corr=slice(-25, 0))
    
#     ax.plot(pw_plot_ds.voltage_cl, pw_plot_ds.delta.values, 'x', label=r'$\theta = {}$'.format(theta))
    
    delta_fit_data = sl_fitter.fit(pw_plot_ds.voltage_cl.values, pw_plot_ds.delta.values)
    delta_0 = delta_fit_data.get_param('y_0')
    delta_0s.append(delta_0)
    
    flattened_delta = (pw_plot_ds.delta.values - delta_0) * np.sqrt(np.sin(np.radians(theta)))
    
    ax.plot(pw_plot_ds.voltage_cl, flattened_delta, 'x', label=r'$\theta = {}$'.format(theta))
    
    delta_fit_data = sl_fitter.fit(pw_plot_ds.voltage_cl, flattened_delta)
    ax.plot(*delta_fit_data.get_fit_plottables(), 
            label=r'$y_0 = {:.3g}$'.format(delta_fit_data.get_param('y_0')))
    ax.legend()


ax.set_ylabel(r'$(\Delta - \Delta_0) \sin^{1/2}{\theta} $')
ax.set_xlabel(r'$|V|^{3/4}$')

delta_0s = np.array(delta_0s)

fig.tight_layout()

In [44]:
print(delta_0s)

print(d_delta_0s)

piecewise_ds.theta.values

[ 3.48223737  8.12342498  9.18608777  8.7844804   9.66204131 10.10223821
  8.68876167]
[1.13982315 0.60212275 0.7579872  0.59098696 0.47533765 0.63424856
 0.59817238]


array([ 3.,  5.,  8., 10., 12., 15., 20.])

In [40]:
fig, ax = plt.subplots()


ax.errorbar(piecewise_ds.theta, 
            delta_0s, label=f'{voltage}', yerr=d_delta_0s)
ax.legend()

In [25]:
# Not a figure from Bergmann

fig, ax = plt.subplots()

for voltage in piecewise_ds.voltage.values:
    ax.plot(np.sqrt(np.sin(np.radians(piecewise_ds.theta.values))), 
            piecewise_ds.sel(voltage=voltage, probe='flat_flush').delta.values - delta_0s, label=f'{voltage}')
    ax.legend()


In [74]:
# Figure 10B

fig, ax = plt.subplots()

plot_ds = piecewise_ds.sel(voltage=-18.0, probe='flat_flush')

ax.plot(np.float_power(np.sin(np.radians(plot_ds.theta.values)), -0.5), plot_ds.delta.values - delta_0s, 'x')


So it looks like the simulations follow the trend of Bergmann's results above ~8 $\degree$.

profiles are very noisy though, potentially implies that the noise is bigger at lower angles